<a href="https://colab.research.google.com/github/vladgap/Various/blob/main/MLN-111221.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import plotly.express as px

# StandardScaler

In [2]:
x=[
[2,1,0],
[-1,0,1],
[0,0,0],
[1,1,0],
[-1,-1,-1],
[-1,1,-1],
]
# x=np.array(x)
scaler = preprocessing.StandardScaler().fit(x)
print(scaler.mean_, scaler.scale_)
x_scaled = scaler.transform(x)
print (x_scaled)
x_back=scaler.inverse_transform(x_scaled)
print (x_back)
scaler.inverse_transform([[0,0,0]])
# help(scaler)

[ 0.          0.33333333 -0.16666667] [1.15470054 0.74535599 0.68718427]
[[ 1.73205081  0.89442719  0.24253563]
 [-0.8660254  -0.4472136   1.69774938]
 [ 0.         -0.4472136   0.24253563]
 [ 0.8660254   0.89442719  0.24253563]
 [-0.8660254  -1.78885438 -1.21267813]
 [-0.8660254   0.89442719 -1.21267813]]
[[ 2.  1.  0.]
 [-1.  0.  1.]
 [ 0.  0.  0.]
 [ 1.  1.  0.]
 [-1. -1. -1.]
 [-1.  1. -1.]]


array([[ 0.        ,  0.33333333, -0.16666667]])

# Single Neuron

In [3]:
class Neuron:
    """A single neuron with an activation function.
       Attributes:
          inputs: The number of inputs in the perceptron, not counting the bias.
          bias:   The bias term. By defaul it's 1.0.
          activ:  The activation function: linear (default), relu, sigmoid."""

    def __init__(self, inputs, bias = 1.0, activ = 'linear'):
        """Return a new Perceptron object with the specified number of inputs (+1 for the bias) and random initial weights.""" 
        self.weights = (np.random.rand(inputs+1) * 2) - 1 
        self.bias = bias
        self.activ = activ

    def run(self, x):
        """Run the perceptron. x is a python list with the input values."""
        sum = np.dot(np.append(x,self.bias),self.weights)
        if self.activ == 'linear':
          return sum
        if self.activ == 'sigmoid':
          return self.sigmoid(sum)
        if self.activ == 'relu':
          return self.relu(sum)
        if self.activ == 'mrelu':
          return self.mrelu(sum)

    def set_weights(self, w_init):
        """Overrides the np.random.rand() weights and the bias weight."""
        # w_init is a list of floats. Organize it as you'd like.
        self.weights=np.array(w_init, dtype='double')

    def set_activ(self, activ, param=0):
        """Overrides the 'linear' activation function."""
        self.activ = activ
        self.param = param

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))
    
    def relu(self, x):
        # return the output of the relu function applied to x
        if x >= 0:
          return x
        return 0

    def mrelu(self, x):
        # return the output of the modified relu function applied to x
        if x >= 0:
          return x
        return self.param*x

In [4]:
neu=Neuron(inputs=2)
neu.set_weights([10,10,-15]) 
neu.set_activ('mrelu', param=0.01)
neu.run([1,0])

-0.05

## AND gate

In [5]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-15]) #AND gate

print("AND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


AND Gate:
0 0 = 0.0000003059
0 1 = 0.0066928509
1 0 = 0.0066928509
1 1 = 0.9933071491


## OR gate

In [6]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-5]) #OR gate

print("OR Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


OR Gate:
0 0 = 0.0066928509
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.9999996941


## NAND gate

In [7]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([-10,-10,15]) #NAND gate

print("NAND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))

NAND Gate:
0 0 = 0.9999996941
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.0066928509


# Multilayer neuron

In [26]:
class MultiLayerNeuron:     
    """A multilayer neuron class that uses the Neuron class above.
       Builds a list of neurons with the specific activation function.
       The activation function may be modified later using the set_activ method.
       For example: mln.network[layer][neuron].set_activ('linear').
       Attributes:
          layers:  A list with the number of neurons per layer. Including the input (first) and the output (last) layers.
          bias:    The bias term. The same bias is used for all neurons.
          activ:   The activation function: linear (default), relu, sigmoid."""

    def __init__(self, layers, bias = 1.0, activ='linear'):
        """Return a new MLP object with the specified parameters.""" 
        self.layers = np.array(layers,dtype=object)
        self.bias = bias
        self.network = [] # The list of lists of neurons (perceptrons).
        self.values = []  # The list of lists of neurons' (perceptrons') output values.
        self.d = []       # The list of lists of error terms (lowercase deltas)
        self.activ = activ

        # 2 nested loops to create neurons layer by layer
        for i in range(len(self.layers)): # outer loop iterates on each layer
            self.values.append([]) #The new list of values will be filled with zeros, for every neuron in the layer. 
            self.values[i] = [0.0 for j in range(self.layers[i])]
            self.d.append([])
            self.d[i] = [0.0 for j in range(self.layers[i])]                        
            self.network.append([])
            if i > 0:      #network[0] is the input layer, so it has no neurons
                for j in range(self.layers[i]): # inner loop iterates on each neuron in a layer
                    neur=Neuron(inputs = self.layers[i-1], bias = self.bias, activ = self.activ) # 
                    self.network[i].append(neur) # adding j perceptrons
        self.network = np.array([np.array(x) for x in self.network],dtype=object) #transforms list of lists to numpy array
        self.values = np.array([np.array(x) for x in self.values],dtype=object)
        self.d = np.array([np.array(x) for x in self.d],dtype=object)

    def set_weights(self, w_init): # set_weights of the MultiLayer class
        """Set the weights. 
           w_init is a list of lists with the weights for all but the input layer."""
        for i in range(len(w_init)):
            for j in range(len(w_init[i])):
                self.network[i+1][j].set_weights(w_init[i][j]) # set_weights for each perceptron i

    def set_activ(self, activ, param=0):
        """Set the activation function to every neurons."""
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                self.network[i][j].set_activ(activ, param) # set_activ for each neuron
        self.param=param
    
    def set_output_activ(self, activ, param=0):
        """Set the activation function to the last (output) neurons."""
        i = len(self.network)-1
        for j in range(self.layers[i]):
            self.network[i][j].set_activ(activ, param) 

    def printWeights(self):
        """Displays a summary of weights and activation functions per layer and neuron."""
        print()
        print('Layer 0 is the Input Layer')
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                print("Layer",i,"Neuron",j,":",self.network[i][j].weights,self.network[i][j].activ)
        print()

    def run(self, x):
        """Feed a sample x into the MultiLayer Neuron."""
        x = np.array(x,dtype=object)
        self.values[0] = x
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):  
                self.values[i][j] = self.network[i][j].run(self.values[i-1]) #runs preceptrons with the previous outputs
        return self.values[-1]

    def bp_classif(self, x, y, eta=0.2):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative of the sigmoid function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        self.eta=eta
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = 2*(y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # ∂MSE/∂weight=∂MSE/∂output*∂output/∂weight
        # STEP 3: Calculate the OUTPUT error terms
        # ∂MSE/∂output -- depends on neuron's activation function
        self.d[-1] = outputs * (1 - outputs) * (error) # derivative of the SIGMOID function 
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k]               
                self.d[i][h] = self.values[i][h] * (1-self.values[i][h]) * fwd_error # derivative of the SIGMOID function
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    if k==self.layers[i-1]:
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] # applying the delta rule
                    self.network[i][j].weights[k] += delta
        return MSE

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))

    def deriv(self, value, i, j=0):
        if self.network[i][j].activ == 'linear':
          # print ('lin')
          return 1
        if self.network[i][j].activ == 'sigmoid':
          # print ('sig')
          return self.sigmoid(value)*(1-self.sigmoid(value))
        if self.network[i][j].activ == 'relu':
          if value > 0:
            # print ('re>')
            return 1
          else:
            # print ('re<')
            return 0
        if self.network[i][j].activ == 'mrelu':
          if value > 0:
            return 1
          else:
            return self.param

    def bp_regres(self, x, y, eta=0.2):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative according each neuron's activation function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        self.eta=eta
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = 2*(y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # ∂MSE/∂weight=∂MSE/∂output*∂output/∂weight
        # STEP 3: Calculate the OUTPUT error terms
        # ∂MSE/∂output -- depends on neuron's activation function
        for j in range (len(outputs)):
            self.d[-1][j] = self.deriv(outputs[j], len(self.network)-1) * error
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k] 
                self.d[i][h] = self.deriv(self.values[i][h], i, h) * fwd_error
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    # output=sum(weight*value)+bias*bias_weight
                    if k==self.layers[i-1]:
                        # ∂output/∂bias_weight=bias
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        # ∂output/∂weight=value
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] 
                    self.network[i][j].weights[k] += delta # applying the delta rule
        return MSE

## XOR gate=(OR+NAND)+AND

In [9]:
#test code
mln1 = MultiLayerNeuron(layers=[2,2,1])  #mln1
mln1.set_weights([[[-10,-10,15],[15,15,-10]],[[10,10,-15]]])
mln1.set_activ('sigmoid') #linear is by default

mln1.printWeights()
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln1.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln1.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln1.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln1.run([1,1])[0]))


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-10. -10.  15.] sigmoid
Layer 1 Neuron 1 : [ 15.  15. -10.] sigmoid
Layer 2 Neuron 0 : [ 10.  10. -15.] sigmoid

XOR Gate:
0 0 = 0.0066958493
0 1 = 0.9923558642
1 0 = 0.9923558642
1 1 = 0.0071528098


## Training 

In [10]:
mln2 = MultiLayerNeuron(layers=[2,2,1])
mln2.set_activ('sigmoid') #linear is by default
print("\nTraining Neural Network as an XOR Gate...\n")
for i in range(2000):
    MSE = 0.0
    MSE += mln2.bp_classif([0,0],[0])
    MSE += mln2.bp_classif([0,1],[1])
    MSE += mln2.bp_classif([1,0],[1])
    MSE += mln2.bp_classif([1,1],[0])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln2.printWeights()
    
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln2.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln2.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln2.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln2.run([1,1])[0]))


Training Neural Network as an XOR Gate...

1.1423002316178406
1.072135076262703
1.0277755561777755
0.7776171017687947
0.727241566885627
0.711280178839184
0.6369062702392623
0.06724173240264072
0.02224606011835311
0.0127782401991423

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-3.9550352  -3.9539741   5.78518035] sigmoid
Layer 1 Neuron 1 : [-6.57406167 -6.63082922  2.43724943] sigmoid
Layer 2 Neuron 0 : [ 7.91780403 -8.13634107 -3.63248563] sigmoid

XOR Gate:
0 0 = 0.0383715751
0 1 = 0.9556781262
1 0 = 0.9553399456
1 1 = 0.0580297584


# Regres

In [88]:
class Regres:
    def __init__(self, layers, bias=1.0):
        self.layers = layers
        self.bias = bias
        self.regres_network = MultiLayerNeuron(layers=layers, bias=bias)

    def set_weights(self, w_init):
        self.regres_network.set_weights(w_init)

    def set_hidden_activ(self, activ, param=0):
        self.regres_network.set_activ(activ, param=0)
        self.regres_network.set_output_activ('linear')

    def fit(self, X, y, epochs, eta=0.2):
        self.epochs=epochs
        self.eta=eta
        self.weight_history=[]
        self.weight_history_table=[]
        self.MSE_history=[]
        for i in range(self.epochs):
            weight_epoch=[]
            weight_epoch_table=[]
            MSE = 0.0
            for j in range (len(X)):
                MSE +=  self.regres_network.bp_regres(X[j],[y[j]], self.eta)
            MSE = MSE / len(X)
            self.MSE_history.append(MSE)
            for m in range(1,len(self.layers)):
                weight_layer=[]
                for n in range(self.layers[m]):
                    neuron_w=self.regres_network.network[m][n].weights
                    neuron_w_list=[x for x in neuron_w]
                    weight_layer.append(neuron_w_list)
                    weight_epoch_table+=neuron_w_list
                weight_epoch.append(weight_layer)
            self.weight_history.append(weight_epoch)
            self.weight_history_table.append(weight_epoch_table)
        self.weight_history_table=pd.DataFrame(data=self.weight_history_table, columns=self.get_cols())
        print ("""Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - pandas table of weights propagation
self.MSE_history - list of MSEs propagation""")

    def get_cols(self): 
        cols=[]
        for i in range(1,len(self.layers)):
            for h in range(self.layers[i]):
                for k in range(self.layers[i-1]+1): 
                    col="{}_{}_{}".format(i,h,k) 
                    cols.append(col)
        return cols 
    
    def printWeights(self):
        self.regres_network.printWeights()

    def run(self, x):
        return self.regres_network.run(x)

    def predict(self, x):
        y=[]
        for i in range(len(x)):
            y.append(self.run(x[i])[0])
        return y

In [12]:
weight_history=[]
a=[[1,2],[1,3],[4,1]]
b=[]
for m in range(3):
  for n in range(2):
    b.append(a[m][n])
print ("b",b)
weight_history.append(b)
print (weight_history)

b [1, 2, 1, 3, 4, 1]
[[1, 2, 1, 3, 4, 1]]


# Regres_scale

In [40]:
# nuzhno podumat' kak vesa obratno perevesti. mozhet inverse_transform
class Regres_scale:
    def __init__(self, layers, bias=1.0):
        self.layers = layers
        self.bias = bias
        self.regres_network = MultiLayerNeuron(layers=layers, bias=bias)

    def set_weights(self, w_init):
        self.regres_network.set_weights(w_init)

    def set_hidden_activ(self, activ):
        self.regres_network.set_activ(activ)
        self.regres_network.set_output_activ('linear')

    def fit(self, x, y, epochs, eta=0.2):
        self.scaler = preprocessing.StandardScaler().fit(x)
        x_scaled=self.scaler.transform(x)
        self.epochs=epochs
        self.eta=eta
        for i in range(epochs):
            MSE = 0.0
            for j in range (len(x)):
                MSE +=  self.regres_network.bp_regres(x_scaled[j],[y[j]])
            MSE = MSE / len(x)
            # dobavit' istoriyu kak v Regres

    def printWeights(self):
        self.regres_network.printWeights()

    def run(self, x):
        return self.regres_network.run(x)

    def predict(self, x):
        y=[]
        for i in range(len(x)):
            y.append(self.run(x[i])[0])
        return y

    def get_standard_params(self, x):
        x2=np.array(x)
        mean=x2.mean(axis=0)
        stdev=x2.std(axis=0)
        return mean, stdev

# Example 1 -- weights

In [14]:
print ("1 1 =",mln1.run([1,1]))
# print (mln1.network[1][0].weights) # network is list of lists of perceptrons. Each has attribute "weights"
w1=mln1.network[1][0].weights
w2=[x for x in w1]
w2

1 1 = [0.00715281]


[-10.0, -10.0, 15.0]

In [29]:
mln1=MultiLayerNeuron(layers=[2,2,1])
mln1.set_activ('relu')
mln1.printWeights()

mln1.set_output_activ('sigmoid') # setting the output activ func
mln1.printWeights()

mln1.network[1][0].set_activ('linear') # changing specific activ func
mln1.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.66718953  0.70649132  0.36681767] relu
Layer 1 Neuron 1 : [-0.99688564 -0.50023133 -0.70730745] relu
Layer 2 Neuron 0 : [-0.92615534  0.57889616 -0.46453555] relu


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.66718953  0.70649132  0.36681767] relu
Layer 1 Neuron 1 : [-0.99688564 -0.50023133 -0.70730745] relu
Layer 2 Neuron 0 : [-0.92615534  0.57889616 -0.46453555] sigmoid


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.66718953  0.70649132  0.36681767] linear
Layer 1 Neuron 1 : [-0.99688564 -0.50023133 -0.70730745] relu
Layer 2 Neuron 0 : [-0.92615534  0.57889616 -0.46453555] sigmoid



# Example 3 -- linear activs, bp_regres. מתכנס למשקלים טוב מאוד 

In [28]:
mln3 = MultiLayerNeuron(layers=[3,1])
# mln3.set_weights([[[5.,-0.9,2.,0.]]])
mln3.set_weights([[[0,0,0,0]]])
mln3.printWeights()
print("\nTraining Neural Network...\n")
for i in range(400):
    MSE = 0.0
    MSE += mln3.bp_regres([0.7759,	0.1104,	0.9977,],5.764286995)
    MSE += mln3.bp_regres([0.9692,	0.6961,	0.8483,],5.84646758)
    MSE += mln3.bp_regres([0.0265,	0.399,	0.5375,],0.808633075)
    MSE += mln3.bp_regres([0.7694,	0.5051,	0.2542,],3.850298589)
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln3.printWeights()




Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0. 0. 0. 0.] linear


Training Neural Network...

41.81534530017634
1.033154313956396e-16

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 4.99973369e+00 -9.99625006e-01  1.99991681e+00  3.52253295e-05] linear



# Example 4 -- sigmoid, bp_regres. מקרב רגרסיה עם סיגמויד. עובד טוב

In [31]:
# generating output values
mln4 = MultiLayerNeuron(layers=[3,2,1])
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')
mln4.set_weights([[[0,1,-2,1],[1,-2,0,1]],[[-1,2,3]]])
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[3,2,1])
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')

for i in range(1000):
    MSE = 0.0
    MSE += mln4.bp_regres([2,1,0],y4[0])
    MSE += mln4.bp_regres([-1,0,1],y4[1])
    MSE += mln4.bp_regres([0,0,0],y4[2])
    MSE += mln4.bp_regres([1,1,0],y4[3])
    MSE += mln4.bp_regres([-1,-1,-1],y4[4])
    MSE += mln4.bp_regres([-1,1,-1],y4[5])
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.  1. -2.  1.] sigmoid
Layer 1 Neuron 1 : [ 1. -2.  0.  1.] sigmoid
Layer 2 Neuron 0 : [-1.  2.  3.] linear



'original values:'

[3.5813200792821274,
 3.731058578630005,
 3.731058578630005,
 3.119202922022118,
 3.880797077977882,
 2.2563920540063265]


MSE= 10.595039118710035
MSE= 0.0020178928467710065
MSE= 0.0011831955981094426
MSE= 0.0006304407963615564
MSE= 0.00036540528847665426

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.89063884 -2.15574041  1.33448787  0.49474532] sigmoid
Layer 1 Neuron 1 : [ 0.52547081 -1.19737947  0.18487482  0.94775626] sigmoid
Layer 2 Neuron 0 : [1.17257518 1.76372824 1.7455312 ] linear



'predicted values:'

[3.5844612056033154,
 3.730045079214485,
 3.745017616069883,
 3.119265134138855,
 3.87913258264512,
 2.2574629251148766]

# Example 5 -- relu-sigmoid, bp_regres. מתכנס למשקלים לא משהו

In [33]:
mln4 = MultiLayerNeuron(layers=[3,2,1,1]) # generating MLN with random weights
mln4.set_activ('relu')
mln4.network[2][0].set_activ('sigmoid') # changing specific activ func
mln4.set_output_activ('linear')
mln4.set_weights([[[0,1,-2,1],[1,-2,0,1]],[[1,0,1]],[[-1,2]]]) # setting weights
mln4.printWeights()

y4=[] # generating output values
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[3,2,1,1]) # generating MLN with random weights
mln4.set_activ('relu')
mln4.network[2][0].set_activ('sigmoid') # changing specific activ func
mln4.set_output_activ('linear')

for i in range(400):
    MSE = 0.0
    MSE += mln4.bp_regres([2,1,0],y4[0])
    MSE += mln4.bp_regres([-1,0,1],y4[1])
    MSE += mln4.bp_regres([0,0,0],y4[2])
    MSE += mln4.bp_regres([1,1,0],y4[3])
    MSE += mln4.bp_regres([-1,-1,-1],y4[4])
    MSE += mln4.bp_regres([-1,1,-1],y4[5])
    MSE = MSE / 6
    if(i%20 == 0):
        print ('MSE', i, '=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.  1. -2.  1.] relu
Layer 1 Neuron 1 : [ 1. -2.  0.  1.] relu
Layer 2 Neuron 0 : [1. 0. 1.] sigmoid
Layer 3 Neuron 0 : [-1.  2.] linear



'original values:'

[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]


MSE 0 = 3.9099573114646997
MSE 20 = 0.04339351861603505
MSE 40 = 0.04218037055054356
MSE 60 = 0.041670323037923084
MSE 80 = 0.040665009486045046
MSE 100 = 0.03843040256064063
MSE 120 = 0.034031047758971276
MSE 140 = 0.026758858196294515
MSE 160 = 0.017449820761076863
MSE 180 = 0.009726905526690907
MSE 200 = 0.005131230751949049
MSE 220 = 0.002650668652175774
MSE 240 = 0.0014979515035392517
MSE 260 = 0.0009467852765953958
MSE 280 = 0.0006522373746369227
MSE 300 = 0.0004927960691656631
MSE 320 = 0.0003990405409014521
MSE 340 = 0.0003416184186920065
MSE 360 = 0.00030447307274159723
MSE 380 = 0.00028034613206168196

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.63526498 0.54161014 0.07359593 0.5617447 ] relu
Layer 1 Neuron 1 : [-0.42972076  0.11272097 -0.94994838  0.75468356] relu
Layer 2 Neuron 0 : [-0.74438857 -0.84037496  0.39255504] sigmoid
Layer 3 Neuron 0 : [0.62605035 0.91491213] linear



'predicted values:'

[1.0406303037083355,
 1.2584334449645556,
 1.1282578163413126,
 1.0522347894544162,
 1.0483198907883362,
 1.0045223069372549]

# Example 7 -- Regres class. עובד באופן עקרוני
# relu בעייתי

In [93]:
reg1=Regres(layers=[3,2,1,1])
reg1.set_weights([[[1,1,0,0],[0,0,0,0]],[[0,0,0]],[[0,0]]])
X=[
[2,1,0],
[-1,0,1],
[0,0,0],
[1,1,0],
[-1,-1,-1],
[-1,1,-1],
]
y=[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]
reg1.set_hidden_activ('relu')
reg1.regres_network.network[2][0].set_activ('sigmoid') # changing specific activ func
reg1.fit(X,y, epochs=5, eta=0.1)
reg1.printWeights()
pred=reg1.predict(X)
print (pred)
print()
# display ('weight_history',reg1.weight_history)
display ('MSE',reg1.MSE_history)

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - pandas table of weights propagation
self.MSE_history - list of MSEs propagation

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [9.99993917e-01 9.99950434e-01 1.19553645e-05 2.60048657e-06] relu
Layer 1 Neuron 1 : [0. 0. 0. 0.] relu
Layer 2 Neuron 0 : [0.00850294 0.         0.03500299] sigmoid
Layer 3 Neuron 0 : [0.42542099 0.84771537] linear

[1.0668595952299427, 1.0641482356911771, 1.06414823804216, 1.0659559852988165, 1.0641482356911771, 1.0641482356911771]



'MSE'

[1.8232957676324437,
 0.10904793902185193,
 0.04350360405968031,
 0.03926659710245326,
 0.03860509075063843]

# Example -- Regres_scale. צריך חשוב ולהשלים

In [92]:
# nuzhno podumat' kak vesa obratno perevesti. mozhet inverse_transform
reg2=Regres_scale(layers=[3,2,1,1])
reg2.set_weights([[[1,1,0,0],[0,0,0,0]],[[0,0,0]],[[0,0]]])
X=[
[2,1,0],
[-1,0,1],
[0,0,0],
[1,1,0],
[-1,-1,-1],
[-1,1,-1],
]
y=[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]
reg2.set_hidden_activ('relu')
reg2.regres_network.network[2][0].set_activ('sigmoid') # changing specific activ func
reg2.fit(X,y,epochs=200, eta=0.1)
reg2.printWeights()
pred=reg2.predict(X)
pred


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.89627003  1.09820987 -0.09844858  0.10980198] relu
Layer 1 Neuron 1 : [0. 0. 0. 0.] relu
Layer 2 Neuron 0 : [-0.20230132  0.         -0.08544344] sigmoid
Layer 3 Neuron 0 : [0.41862384 0.8495285 ] linear



[0.98913380495199,
 1.0499036925246672,
 1.047584393861123,
 1.006481358990087,
 1.0499036925246672,
 1.0412552397208836]

# Example 6 -- https://mmuratarat.github.io/2020-01-09/backpropagation

In [91]:
# generating output values
mln6 = MultiLayerNeuron(layers=[5,3,1])
mln6.set_activ('sigmoid')
mln6.set_output_activ('linear')
mln6.set_weights([[[.19,.33,.4,.51,.54,.1],
                   [.55,.16,.35,.85,.49,.1],
                   [.76,.97,.7,.85,.57,.1]],
                  [[.1,.03,-.17,.1]]
                  ]) # setting initial weights
mln6.printWeights()

# training
for i in range(100):
    MSE = 0.0
    MSE += mln6.bp_regres([.5,.1,1,0,0],[0.1])
    MSE += mln6.bp_regres([.3,.2,0,1,0],[.6])
    MSE += mln6.bp_regres([.7,.9,0,0,1],[.4])
    MSE += mln6.bp_regres([.8,.1,1,0,0],[.1])
    MSE = MSE / 4
    if(i%10 == 0):
        print ('MSE', i, '=',MSE)

mln6.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.19 0.33 0.4  0.51 0.54 0.1 ] sigmoid
Layer 1 Neuron 1 : [0.55 0.16 0.35 0.85 0.49 0.1 ] sigmoid
Layer 1 Neuron 2 : [0.76 0.97 0.7  0.85 0.57 0.1 ] sigmoid
Layer 2 Neuron 0 : [ 0.1   0.03 -0.17  0.1 ] linear

MSE 0 = 0.37156705313996946
MSE 10 = 0.3477538356465295
MSE 20 = 0.2952678357523502
MSE 30 = 0.1929773244127845
MSE 40 = 0.08122195719845282
MSE 50 = 0.026313708157257025
MSE 60 = 0.009140606396217596
MSE 70 = 0.0030643254528590997
MSE 80 = 0.0009964697271421519
MSE 90 = 0.0004154365309422917

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.02456057  0.33840543  0.0334604   0.7232171   0.54268441 -0.05063808] sigmoid
Layer 1 Neuron 1 : [ 0.23164539  0.23135724 -0.27344665  1.21173095  0.55817302 -0.09354269] sigmoid
Layer 1 Neuron 2 : [ 0.89420685  0.80339977  1.20014113  0.31647156  0.44787927 -0.05550803] sigmoid
Layer 2 Neuron 0 : [ 0.492259    0.86010436 -0.75006689  0.11361553] linear



# Example 8 -- Regres_linear. עובד טוב

In [90]:
reg2=Regres(layers=[1,2,1])
X=[[0],[1],[2],[3],[4],[5]]
y=[0,1,2,3,4,5]
reg2.printWeights()

reg2.set_hidden_activ('linear')
reg2.fit(X,y,epochs=5, eta=0.001)
reg2.printWeights()
pred=reg2.predict(X)
print ('predictions',pred)
display ('weight_history_table',reg2.weight_history_table)
display ('MSE',reg2.MSE_history)


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.99413806 0.85782597] linear
Layer 1 Neuron 1 : [0.53722557 0.63382097] linear
Layer 2 Neuron 0 : [ 0.41127724  0.49093749 -0.10033596] linear

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - pandas table of weights propagation
self.MSE_history - list of MSEs propagation

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [1.0196067 0.8597228] linear
Layer 1 Neuron 1 : [0.56660613 0.63605049] linear
Layer 2 Neuron 0 : [ 0.47344819  0.52569982 -0.09585097] linear

predictions [0.645554867264223, 1.4261505530126959, 2.2067462387611685, 2.9873419245096415, 3.7679376102581146, 4.548533296006588]


'weight_history_table'

,1_0_0,1_0_1,1_1_0,1_1_1,2_0_0,2_0_1,2_0_2
0,1.001718,0.859037,0.546210,0.635253,0.431886,0.502593,-0.097441
1,1.007802,0.859743,0.553260,0.636069,0.447299,0.511280,-0.095827
2,1.012648,0.860036,0.558788,0.636402,0.458749,0.517679,-0.095178
3,1.016509,0.860005,0.563142,0.636366,0.467212,0.522341,-0.095249
4,1.019607,0.859723,0.566606,0.636050,0.473448,0.525700,-0.095851


'MSE'

[1.3779776959988277,
 1.0159038856668177,
 0.8143627473587093,
 0.7035760521935384,
 0.6428110133854495]

# Example 9 -- mrelu check. working

MultiLayerNeuron

In [95]:
mln4 = MultiLayerNeuron(layers=[1,2,1]) # generating MLN with random weights
mln4.set_activ('mrelu', param=0.01)
# mln4.set_activ('relu')
mln4.set_output_activ('linear')
mln4.set_weights([[[-1.1,1.74],[3.35,-6.44]],[[-1.41,2.05,2.2]]]) # setting weights
mln4.printWeights()

y4=[] # generating output values
y4.append(mln4.run([0])[0])
y4.append(mln4.run([1])[0])
y4.append(mln4.run([2])[0])
y4.append(mln4.run([3])[0])
y4.append(mln4.run([4])[0])
y4.append(mln4.run([5])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[1,2,1]) # generating MLN with random weights
mln4.set_activ('mrelu', param=0.01)
# mln4.set_activ('relu')
mln4.set_output_activ('linear')

eta=0.01
for i in range(4000):
    MSE = 0.0
    MSE += mln4.bp_regres([0],y4[0], eta)
    MSE += mln4.bp_regres([1],y4[1], eta)
    MSE += mln4.bp_regres([2],y4[2], eta)
    MSE += mln4.bp_regres([3],y4[3], eta)
    MSE += mln4.bp_regres([4],y4[4], eta)
    MSE += mln4.bp_regres([5],y4[5], eta)
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE', i, '=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([0])[0])
y4.append(mln4.run([1])[0])
y4.append(mln4.run([2])[0])
y4.append(mln4.run([3])[0])
y4.append(mln4.run([4])[0])
y4.append(mln4.run([5])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-1.1   1.74] mrelu
Layer 1 Neuron 1 : [ 3.35 -6.44] mrelu
Layer 2 Neuron 0 : [-1.41  2.05  2.2 ] linear



'original values:'

[-0.38541999999999943,
 1.2342550000000005,
 2.7394859999999994,
 9.622496,
 16.505506,
 23.388515999999996]


MSE 0 = 201.16501541426013
MSE 200 = 45.20041172384151
MSE 400 = 44.78297868131754
MSE 600 = 44.68381469935119
MSE 800 = 44.53754477663142
MSE 1000 = 44.2378133205734
MSE 1200 = 43.6209196690275
MSE 1400 = 42.381373233416475
MSE 1600 = 40.01195238177387
MSE 1800 = 35.89310423449275
MSE 2000 = 29.820486041476045
MSE 2200 = 22.843458807158147
MSE 2400 = 16.96895263413609
MSE 2600 = 11.711069086275629
MSE 2800 = 5.434716631484069
MSE 3000 = 2.0315391987116507
MSE 3200 = 0.14798232843561965
MSE 3400 = 0.12085475928538932
MSE 3600 = 0.11193179284048671
MSE 3800 = 0.10348207285526555

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 2.5347297  -4.77633252] mrelu
Layer 1 Neuron 1 : [-10.1575297  -0.3294659] mrelu
Layer 2 Neuron 0 : [  2.31037941 -10.15408818  -0.03961825] linear



'predicted values:'

[-0.11651539372819453,
 0.9734510019444089,
 2.7338793952005718,
 9.621471242649767,
 16.509063090098962,
 23.39665493754816]

# Example 10 - Regres_mrelu. working

In [97]:
reg2=Regres(layers=[1,2,1])
X=[[-3],[-2],[-1],[0],[1],[2],[3],[4],[5]]
y=[9,4,1,0,1,4,9,16,25]
# y=[0,1,2,3,4,5]
reg2.set_hidden_activ('mrelu', param=0.01)
# reg2.set_weights([[[0,0],[0,0]],[[0,0,0]]])
reg2.fit(X,y,epochs=5000, eta=0.0001)
reg2.printWeights()
pred=reg2.predict(X)
print ('predictions',pred)
# display ('weight_history_table',reg2.weight_history_table.iloc[-1])
# display ('MSE',reg2.MSE_history)

fig=px.line(y=reg2.MSE_history[500:])
fig.show()

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - pandas table of weights propagation
self.MSE_history - list of MSEs propagation

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.22986197 -0.69076029] mrelu
Layer 1 Neuron 1 : [-0.51770088 -1.03688919] mrelu
Layer 2 Neuron 0 : [-0.16285638  0.83415195  7.62431378] linear

predictions [8.054914224960092, 7.624313777393424, 7.624313777393424, 7.624313777393424, 7.624313777393424, 7.624313777393424, 7.624313777393424, 7.624313777393424, 7.624313777393424]


In [25]:
history=reg2.weight_history_table.T
# history
fig=px.parallel_coordinates(history)
fig.show()